In [1]:
import random
import numpy as np
import torch

from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding
)
from sklearn.metrics import accuracy_score

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)


C:\Users\areesa\anaconda3\envs\urdu_glue_gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load Dataset (Urdu SST-2)
dataset = load_dataset("sepidmnorozy/Urdu_sentiment")
dataset


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 685
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 294
    })
})

In [3]:
model_name = "xlm-roberta-large"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2
)


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# FREEZE the Transformer
for param in model.base_model.parameters():
    param.requires_grad = False


In [5]:
sum(p.requires_grad for p in model.parameters())


4

In [6]:
# Tokenization Function
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=128
    )


In [7]:
# Tokenize Dataset
tokenized_ds = dataset.map(tokenize_function, batched=True)

tokenized_ds = tokenized_ds.rename_column("label", "labels")
tokenized_ds.set_format("torch")


Map: 100%|██████████████████████████████████████████████████████████████████████████████| 294/294 [00:00<00:00, 15401.17 examples/s]


In [8]:
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    
    acc = accuracy_score(labels, preds)
    f1  = f1_score(labels, preds)
    
    return {
        "accuracy": acc,
        "f1": f1
    }


In [9]:
# Training Arguments
training_args = TrainingArguments(
    output_dir="./sst2_xlmr_zero_shot",
    eval_strategy="epoch",
    save_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=20,
    weight_decay=0.01,
    warmup_ratio=0.1,
    logging_steps=50,
    seed=SEED,
    fp16=torch.cuda.is_available()
)


In [10]:
# Trainer
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


C:\Users\areesa\AppData\Local\Temp\ipykernel_69712\3038989936.py:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [11]:
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.696385,0.513605,0.000000
2,No log,0.692556,0.520408,0.053691
3,0.699800,0.692481,0.513605,0.000000
4,0.699800,0.690900,0.517007,0.013889
5,0.695000,0.690632,0.544218,0.141026
6,0.695000,0.690204,0.551020,0.185185
7,0.690900,0.691008,0.517007,0.639594
8,0.690900,0.690732,0.564626,0.609756
9,0.690900,0.689925,0.564626,0.255814
10,0.696300,0.689613,0.513605,0.013793


TrainOutput(global_step=440, training_loss=0.6937230760400945, metrics={'train_runtime': 15.3426, 'train_samples_per_second': 892.941, 'train_steps_per_second': 28.678, 'total_flos': 1172169435437784.0, 'train_loss': 0.6937230760400945, 'epoch': 20.0})

In [12]:
results = trainer.evaluate()
results


{'eval_loss': 0.6885197162628174,
 'eval_accuracy': 0.5578231292517006,
 'eval_f1': 0.25287356321839083,
 'eval_runtime': 0.2345,
 'eval_samples_per_second': 1253.743,
 'eval_steps_per_second': 42.644,
 'epoch': 20.0}